In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

In [ ]:
filename_grokKey = "..\\GroqAPIKey.txt"

In [6]:
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### RunnablePassthrough, RunnableLambda and RunnableParallel

In [7]:
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableParallel
from langchain_groq import ChatGroq

In [8]:
explainer_template = PromptTemplate(
    input_variables=["topic"],
    template="Explain this topic in a single paragraph as if you were explaining it to a 10 year old - {topic}"
)

In [9]:
gemini_llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')

In [10]:
gemini_chain = explainer_template | gemini_llm | StrOutputParser()
print(gemini_chain.invoke({"topic":"Gravity"}))

Imagine you have a ball in your hand. When you let go, it falls to the ground, right? That's because of something called gravity! It's an invisible force that pulls everything towards each other. The Earth is really big and heavy, so it has strong gravity that pulls everything towards it, like us, the ball, and even the air we breathe. That's why we stay on the ground and don't float away! 


In [11]:
lamma_llm = ChatGroq(model="llama-3.1-8b-instant")

In [12]:
lamma_chain = explainer_template | lamma_llm | StrOutputParser()
print(lamma_chain.invoke({"topic":"Gravity"}))

Gravity is a really cool force that pulls everything towards each other. Imagine you have a ball, and you throw it up in the air. What happens? It comes back down to the ground, right? That's because the Earth is pulling on the ball with its gravity. The Earth is like a big magnet that attracts everything towards it, but instead of using a magnetic field, it uses gravity. The more massive the object, the stronger its gravity is. That's why you don't float off into space when you're standing on the Earth's surface - the Earth's gravity is pulling you down. It's also what keeps you on the ground when you're standing or sitting. So, gravity is like a big hug from the Earth that keeps everything connected to it.


In [13]:
analyzer_template = PromptTemplate(
    input_variables=["topic","explanation1","explanation2"],
    template="""
            Which of the two explanations given below are better for explaining {topic} to 10 year old students.
            ###
            Explanation 1 - {explanation1}
            ###
            ***
            Explanation 2 - {explanation2}
            """
)

In [16]:
combined_chain = (
    RunnableParallel({"explanation1":gemini_chain, "explanation2":lamma_chain, "topic":RunnablePassthrough()}) 
    | analyzer_template | gemini_llm | StrOutputParser() 
)

In [18]:
print(combined_chain.invoke({"topic":"Gravity"}))

Both explanations are good and age-appropriate, but **Explanation 1 is slightly better for a 10-year-old** for these reasons:

* **Clearer Structure:** Explanation 1 follows a more logical flow, starting with a relatable example (dropping a ball) and then introducing the concept of gravity.  It then expands to explain Earth's gravity and its effects. 
* **Emphasis on Size:**  Explanation 1 clearly connects the strength of gravity to the size of the object, which is a fundamental concept.
* **Concise and Focused:** Explanation 1 stays focused on the core concept of gravity and its effects.  

While Explanation 2 is engaging and uses good analogies, it's slightly less focused and might be a little harder for a 10-year-old to follow all the points at once.

**Here's how you could make Explanation 1 even better:**

* **Visual Aid:**  Pair the explanation with a visual, like a picture of the Earth and moon or a ball falling to the ground.
* **Interactive Element:**  Have the child drop diff

In [21]:
analyzer_chain = analyzer_template | gemini_llm | StrOutputParser()

updated_chain = (
    RunnableParallel({"explanation1":gemini_chain, "explanation2":lamma_chain, "topic":RunnablePassthrough()})
    | RunnableParallel({"input": RunnablePassthrough(), "analysis":analyzer_chain})
)


In [22]:
updated_chain.invoke({"topic":"Gravity"})

{'input': {'explanation1': "Imagine you throw a ball up in the air.  What happens? It comes back down, right? That's because of something called gravity! It's an invisible force that pulls everything towards each other. The Earth is really big, so it has strong gravity that pulls everything towards the ground, like you, me, that ball, and even the air we breathe! The sun has even stronger gravity, which is why the Earth stays close and goes around it. ",
  'explanation2': "Gravity is a really cool thing that happens in our universe. It's like a magic string that pulls everything towards each other. Imagine you have a ball and you throw it up in the air. What happens? It comes back down to the ground, right? That's because the Earth is pulling on the ball with its gravity. The Earth is like a giant magnet, but instead of making things stick to it, it pulls everything towards itself. That's why you don't float off into space when you're standing on the ground. It's also why things fall d